In [1]:
import cv2
import numpy as np

class BicubicUpscaler:
    def __init__(self, scale_factor=4):
        """
        Initialize the bicubic upscaler.
        
        Args:
            scale_factor (int): Scaling factor for the image (default: 2)
        """
        self.scale_factor = scale_factor
        self.t = 0.5
        self._compute_coefficients()
    
    def _compute_coefficients(self):
        """Compute the bicubic interpolation coefficients"""
        tt = self.t ** 2
        ttt = self.t ** 3
        
        self.q1 = (-ttt + 2*tt - self.t) / 2
        self.q2 = (3*ttt - 5*tt + 2) / 2
        self.q3 = (-3*ttt + 4*tt + self.t) / 2
        self.q4 = (ttt - tt) / 2
    
    def _interpolate_pixels(self, p1, p2, p3, p4):
        """
        Interpolate between four pixels using bicubic coefficients.
        
        Args:
            p1, p2, p3, p4: Four adjacent pixels
        
        Returns:
            float: Interpolated pixel value
        """
        return round(p1 * self.q1 + p2 * self.q2 + p3 * self.q3 + p4 * self.q4)
    
    def upscale(self, input_file, output_file=None):
        """
        Upscale an image using bicubic interpolation.
        
        Args:
            input_file (str): Path to input image
            output_file (str, optional): Path to save output image
            
        Returns:
            numpy.ndarray: Upscaled image array
        """
        try:
            # Read image
            image = cv2.imread(input_file)
            if image is None:
                raise ValueError(f"Could not read image from {input_file}")
            
            # Get dimensions
            height, width = image.shape[:2]
            
            # Split channels
            b, g, r = cv2.split(image)
            
            # Initialize upscaled channel arrays
            new_height = int(height * self.scale_factor)
            new_width = int(width * self.scale_factor)
            
            r_upscale = np.zeros((new_height, new_width))
            g_upscale = np.zeros((new_height, new_width))
            b_upscale = np.zeros((new_height, new_width))
            
            # Step 1: Copy original pixels
            for i in range(0, new_height, 2):
                for j in range(0, new_width, 2):
                    r_upscale[i, j] = r[i//2, j//2]
                    g_upscale[i, j] = g[i//2, j//2]
                    b_upscale[i, j] = b[i//2, j//2]
            
            # Step 2: Interpolate horizontally
            for i in range(0, new_height, 2):
                for j in range(1, new_width-1, 2):
                    if j < 2 or j >= new_width - 3:
                        # Edge case: simple copy
                        r_upscale[i, j] = r_upscale[i, j-1]
                        g_upscale[i, j] = g_upscale[i, j-1]
                        b_upscale[i, j] = b_upscale[i, j-1]
                    else:
                        # Bicubic interpolation
                        r_upscale[i, j] = self._interpolate_pixels(
                            r_upscale[i, j-3], r_upscale[i, j-1],
                            r_upscale[i, j+1], r_upscale[i, j+3]
                        )
                        g_upscale[i, j] = self._interpolate_pixels(
                            g_upscale[i, j-3], g_upscale[i, j-1],
                            g_upscale[i, j+1], g_upscale[i, j+3]
                        )
                        b_upscale[i, j] = self._interpolate_pixels(
                            b_upscale[i, j-3], b_upscale[i, j-1],
                            b_upscale[i, j+1], b_upscale[i, j+3]
                        )
            
            # Step 3: Interpolate vertically
            for j in range(0, new_width):
                for i in range(1, new_height-1, 2):
                    if i < 2 or i >= new_height - 3:
                        # Edge case: simple copy
                        r_upscale[i, j] = r_upscale[i-1, j]
                        g_upscale[i, j] = g_upscale[i-1, j]
                        b_upscale[i, j] = b_upscale[i-1, j]
                    else:
                        # Bicubic interpolation
                        r_upscale[i, j] = self._interpolate_pixels(
                            r_upscale[i-3, j], r_upscale[i-1, j],
                            r_upscale[i+1, j], r_upscale[i+3, j]
                        )
                        g_upscale[i, j] = self._interpolate_pixels(
                            g_upscale[i-3, j], g_upscale[i-1, j],
                            g_upscale[i+1, j], g_upscale[i+3, j]
                        )
                        b_upscale[i, j] = self._interpolate_pixels(
                            b_upscale[i-3, j], b_upscale[i-1, j],
                            b_upscale[i+1, j], b_upscale[i+3, j]
                        )
            
            # Step 4: Interpolate remaining pixels
            for i in range(1, new_height-1, 2):
                for j in range(1, new_width-1, 2):
                    if i < 3 or i >= new_height - 4 or j < 3 or j >= new_width - 4:
                        # Edge case: simple copy
                        r_upscale[i, j] = r_upscale[i-1, j]
                        g_upscale[i, j] = g_upscale[i-1, j]
                        b_upscale[i, j] = b_upscale[i-1, j]
                    else:
                        # Bicubic interpolation
                        r_upscale[i, j] = self._interpolate_pixels(
                            r_upscale[i-3, j], r_upscale[i-1, j],
                            r_upscale[i+1, j], r_upscale[i+3, j]
                        )
                        g_upscale[i, j] = self._interpolate_pixels(
                            g_upscale[i-3, j], g_upscale[i-1, j],
                            g_upscale[i+1, j], g_upscale[i+3, j]
                        )
                        b_upscale[i, j] = self._interpolate_pixels(
                            b_upscale[i-3, j], b_upscale[i-1, j],
                            b_upscale[i+1, j], b_upscale[i+3, j]
                        )
            
            # Merge channels
            upscaled_image = cv2.merge([b_upscale, g_upscale, r_upscale])
            
            # Save if output path is provided
            if output_file:
                cv2.imwrite(output_file, upscaled_image)
                print(f"Saved upscaled image to {output_file}")
            
            return upscaled_image
            
        except Exception as e:
            print(f"Error during upscaling: {str(e)}")
            raise




In [ ]:
"""# Or process multiple images
image_files = ["image1.jpg", "image2.jpg", "image3.jpg"]
for img_file in image_files:
    output_file = f"upscaled_{img_file}"
    upscaler.upscale(img_file, output_file)

In [2]:
import cv2
import numpy as np

class BicubicUpscaler:
    def __init__(self, scale_factor=4):
        """
        Initialize the bicubic upscaler.
        
        Args:
            scale_factor (int): Scaling factor for the image (default: 4)
        """
        self.scale_factor = scale_factor
        self.t = 0.5
        self._compute_coefficients()
    
    def _compute_coefficients(self):
        """Compute the bicubic interpolation coefficients"""
        tt = self.t ** 2
        ttt = self.t ** 3
        
        self.q1 = (-ttt + 2*tt - self.t) / 2
        self.q2 = (3*ttt - 5*tt + 2) / 2
        self.q3 = (-3*ttt + 4*tt + self.t) / 2
        self.q4 = (ttt - tt) / 2
    
    def _interpolate_pixels(self, p1, p2, p3, p4):
        """
        Interpolate between four pixels using bicubic coefficients.
        
        Args:
            p1, p2, p3, p4: Four adjacent pixels
        
        Returns:
            float: Interpolated pixel value
        """
        return np.clip(round(p1 * self.q1 + p2 * self.q2 + p3 * self.q3 + p4 * self.q4), 0, 255)
    
    def upscale(self, input_file, output_file=None):
        """
        Upscale an image using bicubic interpolation.
        
        Args:
            input_file (str): Path to input image
            output_file (str, optional): Path to save output image
            
        Returns:
            numpy.ndarray: Upscaled image array
        """
        try:
            # Read image
            image = cv2.imread(input_file)
            if image is None:
                raise ValueError(f"Could not read image from {input_file}")
            
            # Get dimensions
            height, width = image.shape[:2]
            
            # Split channels
            b, g, r = cv2.split(image)
            
            # Initialize upscaled channel arrays
            new_height = height * self.scale_factor
            new_width = width * self.scale_factor
            
            r_upscale = np.zeros((new_height, new_width))
            g_upscale = np.zeros((new_height, new_width))
            b_upscale = np.zeros((new_height, new_width))
            
            # Step 1: Copy original pixels
            for i in range(0, new_height, self.scale_factor):
                for j in range(0, new_width, self.scale_factor):
                    r_upscale[i, j] = r[i//self.scale_factor, j//self.scale_factor]
                    g_upscale[i, j] = g[i//self.scale_factor, j//self.scale_factor]
                    b_upscale[i, j] = b[i//self.scale_factor, j//self.scale_factor]
            
            # Step 2: Interpolate horizontally for each row containing original pixels
            for i in range(0, new_height, self.scale_factor):
                for j in range(self.scale_factor, new_width - self.scale_factor, self.scale_factor):
                    # Calculate intermediate points between original pixels
                    for k in range(self.scale_factor):
                        if j-self.scale_factor < 0 or j+self.scale_factor >= new_width:
                            # Edge case: simple linear interpolation
                            factor = (k + 1) / self.scale_factor
                            r_upscale[i, j-self.scale_factor+k] = (1-factor) * r_upscale[i, j-self.scale_factor] + factor * r_upscale[i, j]
                            g_upscale[i, j-self.scale_factor+k] = (1-factor) * g_upscale[i, j-self.scale_factor] + factor * g_upscale[i, j]
                            b_upscale[i, j-self.scale_factor+k] = (1-factor) * b_upscale[i, j-self.scale_factor] + factor * b_upscale[i, j]
                        else:
                            # Bicubic interpolation
                            r_upscale[i, j-self.scale_factor+k] = self._interpolate_pixels(
                                r_upscale[i, j-2*self.scale_factor], r_upscale[i, j-self.scale_factor],
                                r_upscale[i, j], r_upscale[i, j+self.scale_factor]
                            )
                            g_upscale[i, j-self.scale_factor+k] = self._interpolate_pixels(
                                g_upscale[i, j-2*self.scale_factor], g_upscale[i, j-self.scale_factor],
                                g_upscale[i, j], g_upscale[i, j+self.scale_factor]
                            )
                            b_upscale[i, j-self.scale_factor+k] = self._interpolate_pixels(
                                b_upscale[i, j-2*self.scale_factor], b_upscale[i, j-self.scale_factor],
                                b_upscale[i, j], b_upscale[i, j+self.scale_factor]
                            )
            
            # Step 3: Interpolate vertically between the interpolated rows
            for j in range(new_width):
                for i in range(self.scale_factor, new_height - self.scale_factor, self.scale_factor):
                    # Calculate intermediate points between rows
                    for k in range(self.scale_factor):
                        if i-self.scale_factor < 0 or i+self.scale_factor >= new_height:
                            # Edge case: simple linear interpolation
                            factor = (k + 1) / self.scale_factor
                            r_upscale[i-self.scale_factor+k, j] = (1-factor) * r_upscale[i-self.scale_factor, j] + factor * r_upscale[i, j]
                            g_upscale[i-self.scale_factor+k, j] = (1-factor) * g_upscale[i-self.scale_factor, j] + factor * g_upscale[i, j]
                            b_upscale[i-self.scale_factor+k, j] = (1-factor) * b_upscale[i-self.scale_factor, j] + factor * b_upscale[i, j]
                        else:
                            # Bicubic interpolation
                            r_upscale[i-self.scale_factor+k, j] = self._interpolate_pixels(
                                r_upscale[i-2*self.scale_factor, j], r_upscale[i-self.scale_factor, j],
                                r_upscale[i, j], r_upscale[i+self.scale_factor, j]
                            )
                            g_upscale[i-self.scale_factor+k, j] = self._interpolate_pixels(
                                g_upscale[i-2*self.scale_factor, j], g_upscale[i-self.scale_factor, j],
                                g_upscale[i, j], g_upscale[i+self.scale_factor, j]
                            )
                            b_upscale[i-self.scale_factor+k, j] = self._interpolate_pixels(
                                b_upscale[i-2*self.scale_factor, j], b_upscale[i-self.scale_factor, j],
                                b_upscale[i, j], b_upscale[i+self.scale_factor, j]
                            )
            
            # Merge channels and ensure proper data type
            upscaled_image = cv2.merge([b_upscale.astype(np.uint8), 
                                      g_upscale.astype(np.uint8), 
                                      r_upscale.astype(np.uint8)])
            
            # Save if output path is provided
            if output_file:
                cv2.imwrite(output_file, upscaled_image)
                print(f"Saved upscaled image to {output_file}")
            
            return upscaled_image
            
        except Exception as e:
            print(f"Error during upscaling: {str(e)}")
            raise

In [9]:
upscaler = BicubicUpscaler(scale_factor=4)

# Upscale single image
upscaler.upscale(
    input_file=r"C:\Users\91995\OneDrive\Desktop\ESCPN++\TestImg\Zoomed\Set5_hr_zoomed_region.png",
    output_file=r"C:\Users\91995\OneDrive\Desktop\ESCPN++\TestImg\BiCubic\BiCubic_Set5_hr_zoomed_region.png"
)


Saved upscaled image to C:\Users\91995\OneDrive\Desktop\ESCPN++\TestImg\BiCubic\BiCubic_Set5_hr_zoomed_region.png


array([[[105, 192, 236],
        [108, 198, 241],
        [108, 198, 241],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[106, 194, 237],
        [109, 200, 242],
        [109, 200, 242],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[106, 194, 237],
        [109, 200, 242],
        [109, 200, 242],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]]

"C:\Users\91995\OneDrive\Desktop\ESCPN++\TestImg\Zoomed\Set14_hr_zoomed_region.png"